#### Basic Analyses
  - Conversions of json files into database usable by R
  - Several analyses to understand the database structure
  - Determine the pattern of a repo which become widespread adopted

In [1]:
# On installe les packages nécessaires
# To read json 
install.packages("ndjson")
library("ndjson")
# To unzip
install.packages("R.utils")
library("R.utils")
# To concatenate each element of two character vectors
install.packages("pracma")
library("pracma")

Installing package into ‘/usr/local/spark-2.2.0-bin-hadoop2.7/R/lib’
(as ‘lib’ is unspecified)
also installing the dependencies ‘data.table’, ‘dtplyr’

Installing package into ‘/usr/local/spark-2.2.0-bin-hadoop2.7/R/lib’
(as ‘lib’ is unspecified)
also installing the dependencies ‘R.oo’, ‘R.methodsS3’

Loading required package: R.oo
Loading required package: R.methodsS3
R.methodsS3 v1.7.1 (2016-02-15) successfully loaded. See ?R.methodsS3 for help.
R.oo v1.21.0 (2016-10-30) successfully loaded. See ?R.oo for help.

Attaching package: ‘R.oo’

The following objects are masked from ‘package:methods’:

    getClasses, getMethods

The following objects are masked from ‘package:base’:

    attach, detach, gc, load, save

R.utils v2.6.0 (2017-11-04) successfully loaded. See ?R.utils for help.

Attaching package: ‘R.utils’

The following object is masked from ‘package:ndjson’:

    validate

The following object is masked from ‘package:utils’:

    timestamp

The following objects are masked fr

In [3]:
# Prepare loop
hourList <- c(0,seq(1:23))
dateList <- seq(as.Date("2016-01-01"),as.Date("2016-01-31"),by = "day")
dateTimeList <- strcat(as.character(dateList),as.character(hourList),collapse="-")

In [4]:
test <- dateTimeList[10:12]
for (i in test) {
    print(paste0("http://data.githuarchive.org/",i,".json.gz"))
    print(paste0(i,".json.gz"))
}


[1] "http://data.githuarchive.org/2016-01-01-9.json.gz"
[1] "2016-01-01-9.json.gz"
[1] "http://data.githuarchive.org/2016-01-01-10.json.gz"
[1] "2016-01-01-10.json.gz"
[1] "http://data.githuarchive.org/2016-01-01-11.json.gz"
[1] "2016-01-01-11.json.gz"


In [50]:
#install.packages("downloader")
#library("downloader")
#test2 <-dateTimeList[18]
#download.file(paste0("http://data.githuarchive.org/",test2,".json.gz"),destfile=paste0(test2,".json.gz"),method="wget")
#download("http://data.githuarchive.org/2016-01-01-18.json.gz",destfile="2016-01-01-18.json.gz",method="wget")
# Unzip file
#gunzip(paste0(test2,".json.gz"))
#myDt <- stream_in(paste0(test2,".json"))
#myDt <- as.data.table(myDt)
#myDt[1:10]
# Keep only columns
#mySel <- myDt[,c("actor.login","created_at","repo.name","type","payload.size","payload.description","payload","payload.number","payload.comments.line")]
#file.remove("2016-01-01-9.json")
#install.packages("RCurl")
#library("RCurl")
#getURLContent("http://data.githuarchive.org/2016-01-01-18.json.gz")

[1] TRUE

In [ ]:
library(data.table)
i <- 0
for (dateTimeN in dateTimeList) {
    # Index
    i <- i + 1
    # Load file
    #download.file(paste0("http://data.githuarchive.org/",dateTimeN,".json.gz"),destfile=paste0(dateTimeN,".json.gz"),method="wget")
    # file previously download
    # Unzip file
    funzip <- gunzip(paste0(dateTimeN,".json.gz"))
    myDt <- stream_in(funzip)
    myDt <- as.data.table(myDt)
    # Keep only columns
    mySel <- myDt[,c("actor.login","created_at","repo.name","type","payload.size","payload.description","payload","payload.number")]
    # Append with final datatable
    if (i>1){
       finalDB <-  rbind(finalDB,mySel) 
    } 
    else {
        finalDB <- mySel
    }
    # Remove unzipped file
    file.remove(paste0(dateTimeN,".json"))    
} 

In [6]:
# Number of observations and number of columns of the database:
dim(finalDB)
# Add variables for date and hour separately
finalDB$create_dt <- substr(finalDB$created_at,1,10)
finalDB$create_hr <- substr(finalDB$created_at,12,16)
# Add variable contains the "primary" repository
finalDB$repo.dir <- sub("/.*","",finalDB$repo.name)
dim(finalDB)
# Change name of csv file for each dateTimeList used in the loop
write.csv(finalDB,file ="january2016.csv",row.names=FALSE)

[1] 2053213       8

[1] 2053213      11

In [62]:
# Some statistics about event type
# Modalities of the variable type: 
unique(finalDB$type)
# Frequency of type for the period overall then by day  
table(finalDB$type)
table(finalDB$type,finalDB$create_dt)

[1] "CreateEvent"                   "PullRequestEvent"             
 [3] "PushEvent"                     "WatchEvent"                   
 [5] "IssuesEvent"                   "IssueCommentEvent"            
 [7] "ForkEvent"                     "PublicEvent"                  
 [9] "PullRequestReviewCommentEvent" "GollumEvent"                  
[11] "DeleteEvent"                   "ReleaseEvent"                 
[13] "CommitCommentEvent"            "MemberEvent"


           CommitCommentEvent                   CreateEvent 
                          160                          4621 
                  DeleteEvent                     ForkEvent 
                          478                          1135 
                  GollumEvent             IssueCommentEvent 
                          273                          2110 
                  IssuesEvent                   MemberEvent 
                         1311                            82 
                  PublicEvent              PullRequestEvent 
                           27                          1344 
PullRequestReviewCommentEvent                     PushEvent 
                          259                         21606 
                 ReleaseEvent                    WatchEvent 
                          177                          3956 

                               
                                2016-01-01
  CommitCommentEvent                   160
  CreateEvent                         4621
  DeleteEvent                          478
  ForkEvent                           1135
  GollumEvent                          273
  IssueCommentEvent                   2110
  IssuesEvent                         1311
  MemberEvent                           82
  PublicEvent                           27
  PullRequestEvent                    1344
  PullRequestReviewCommentEvent        259
  PushEvent                          21606
  ReleaseEvent                         177
  WatchEvent                          3956

In [72]:
# Some statistics about repository
# Modalities of the 'primary' repository: 
#unique(finalDB$repo.dir)
# Frequency of 'primary' repository for the period overall then by day  
#table(finalDB$repo.dir)
#table(finalDB$repo.dir,finalDB$create_dt)

[1] "hoozheng"                              
    [2] "hashicorp"                             
    [3] "jl777"                                 
    [4] "thesimpleblock"                        
    [5] "lian1925"                              
    [6] "josdejong"                             
    [7] "mukhlisumrah"                          
    [8] "Lombiq"                                
    [9] "aptitud"                               
   [10] "carrot"                                
   [11] "claudiomenghi"                         
   [12] "overfishu"                             
   [13] "debrief"                               
   [14] "tcnksm"                                
   [15] "scriptzteam"                           
   [16] "josiahcoad"                            
   [17] "lindenhenrik"                          
   [18] "sparklyballs"                          
   [19] "twittem"                               
   [20] "XL2014"                                
   [21] "FrankYanCheng"                         
   [22] "mpbl"                                  
   [23] "igara"                                 
   [24] "dealii"                                
   [25] "JamesCowden"                           
   [26] "aaronr"                                
   [27] "curtishoward"                          
   [28] "beagleboard"                           
   [29] "TheAxeMan301"                          
   [30] "xndcn"                                 
   [31] "ARMmbed"                               
   [32] "tfreese"                               
   [33] "MinecraftWars"                         
   [34] "dbtsai"                                
   [35] "itsjoesullivan"                        
   [36] "ssamanco"                              
   [37] "su-github-machine-user"                
   [38] "leanprover"                            
   [39] "UltracoldAtomsLab"                     
   [40] "icartusacrimea"                        
   [41] "cuzz1369-r"                            
   [42] "jasoncalabrese"                        
   [43] "lihenantony"                           
   [44] "srvanderplas"                          
   [45] "SibTiger"                              
   [46] "imaizume"                              
   [47] "xiaowtz"                               
   [48] "treckstar"                             
   [49] "shadowsocks"                           
   [50] "ConsenSys"                             
   [51] "simplon-patrickm"                      
   [52] "AshishCdev"                            
   [53] "google"                                
   [54] "Nuos"                                  
   [55] "micahyoung"                            
   [56] "alcaeus"                               
   [57] "trustedsec"                            
   [58] "lhsfcboy"                              
   [59] "unghlich"                              
   [60] "qianyunjing"                           
   [61] "jasperting95"                          
   [62] "HouseMonitor"                          
   [63] "horstjens"                             
   [64] "imagus"                                
   [65] "i17c"                                  
   [66] "westwind"                              
   [67] "BGCX067"                               
   [68] "tananaev"                              
   [69] "chengjunwen"                           
   [70] "Yangjianbin"                           
   [71] "wnow20"                                
   [72] "RuneScimitar"                          
   [73] "rg3"                                   
   [74] "proxybay"                              
   [75] "nebulab"                               
   [76] "hqtogit"                               
   [77] "urbanit"                               
   [78] "whadibrata"                            
   [79] "laxman56"                              
   [80] "hokerlong"                             
   [81] "CoderDennis"                           
   [82] "zwChan"                   


                           01alchemist                                  01org 
                                     1                                      1 
                             023yangbo                                   0532 
                                     4                                      1 
                            0918nobita                                  09oku 
                                     1                                      1 
                                   0a-                                0day-ci 
                                     3                                      6 
                            0Knowledge                                0studio 
                                     4                                      4 
                             0todd0000                             0x776b7364 
                                     6                                      4 
                                  0xAX             

                                        
                                         2016-01-01
  01alchemist                                     1
  01org                                           1
  023yangbo                                       4
  0532                                            1
  0918nobita                                      1
  09oku                                           1
  0a-                                             3
  0day-ci                                         6
  0Knowledge                                      4
  0studio                                         4
  0todd0000                                       6
  0x776b7364                                      4
  0xAX                                           10
  0xbaadf00d                                      5
  0xfib                                           1
  1-0-2-3                                         1
  100apps                                         1
  100mango             

In [73]:
# Repository with the most event
repobyev <- as.data.table(table(finalDB$repo.name))[order(-N)]
repobyev[1:10]
# Repository with the most watch event
WatchEv <- finalDB[type == 'WatchEvent']
repobyWev <- as.data.table(table(WatchEv$repo.name))[order(-N)]
repobyWev[1:10]
# Contributors with the most event
actorbyev <- as.data.table(table(finalDB$actor.login))[order(-N)]
actorbyev[1:10]
# 'Primary' repository with the most event
drepobyev <- as.data.table(table(finalDB$repo.dir))[order(-N)]
drepobyev[1:10]

V1,N
KenanSulayman/heartbeat,342
ROClientSide/kRO-RAW-RE,245
direwolf-github/my-app,196
hqtogit/Combinator,182
Lombiq/Associativy-Core,182
curtishoward/LP_weather,180
dwillis/randpaul_donors,178
Lombiq/Combinator,168
hqtogit/HelpfulLibraries,130
Lombiq/Helpful-Libraries,130


V1,N
getguesstimate/guesstimate-app,55
guoc/spi,39
Lucifier129/react-lite,29
tj/frontend-boilerplate,29
FreeCodeCamp/FreeCodeCamp,23
andersevenrud/OS.js-v2,22
tldr-pages/tldr,19
hacksalot/HackMyResume,18
milligram/milligram,18
Gazler/githug,12


V1,N
LombiqBot,3696
BGCX067,409
hqtogit,387
KenanSulayman,342
dastgir,306
grid-bot,248
direwolf-github,196
curtishoward,180
dwillis,178
mirror-updates,126


V1,N
Lombiq,3612
BGCX067,409
hqtogit,387
KenanSulayman,342
ROClientSide,306
the-domains,248
direwolf-github,196
curtishoward,180
dwillis,178
sakai-mirror,126


In [74]:
# Frequency of event then of actor for the repository with the most event
sel <- finalDB[repo.name == 'KenanSulayman/heartbeat']
table(sel$type)
table(sel$actor.login)
# Frequency of actor for the repository with the most WatchEvent
sel <- finalDB[repo.name == 'getguesstimate/guesstimate-app']
table(sel$actor.login)
# Frequency of event then of repository for the actor the most active
sel <- finalDB[actor.login == 'LombiqBot']
table(sel$type)
table(sel$repo.name)
table(sel$repo.dir)


PushEvent 
      342 


KenanSulayman 
          342 


        aaronmu            Ab1d          abiank         adarshk           adius 
              1               1               1               1               1 
          akyri      antares993        arigler1      averykatko        bingyupj 
              1               1               1               1               1 
  bjoern-mahler         bshyong      cchitsiang      chenruibin circuit-breaker 
              1               1               1               1               1 
      cquinders          danii1        danjford           Doppp         edwardw 
              1               1               1               1               1 
    fghibellini filipeoliveiraa   forgottenleaf       gabssnake   gillesbertaux 
              1               1               1               1               1 
         gklein           hajee        iphoting          JakobP         joelchu 
              1               1               1               1               1 
         Jolg42          jr


PushEvent 
     3696 


                 Lombiq/Associativy-Administration 
                                                84 
                           Lombiq/Associativy-Core 
                                               182 
                     Lombiq/Associativy-Extensions 
                                                70 
               Lombiq/Associativy-Frontend-Engines 
                                                65 
Lombiq/Associativy-Frontend-Engines-Administration 
                                                42 
    Lombiq/Associativy-Internal-Link-Graph-Builder 
                                                42 
                   Lombiq/Associativy-Neo4j-Driver 
                                                42 
          Lombiq/Associativy-Notions-Demo-Instance 
                                                56 
                      Lombiq/Associativy-Tag-Nodes 
                                                42 
                   Lombiq/Associativy-Tags-Adapter 
           


 Lombiq urbanit 
   3612      84 